In [8]:
"""
Estimate Relaxation/Concentration from Band Powers

This Notebook shows how to buffer, epoch, and transform EEG data from a single
electrode into values for each of the classic frequencies (e.g. alpha, beta, theta)
Furthermore, it generates a neurofeedback metric based on these band powers.

"""

import numpy as np  # Module that simplifies computations on matrices
import matplotlib.pyplot as plt  # Module used for plotting
from pylsl import StreamInlet, resolve_byprop  # Module to receive EEG data
import utils  # Our own utility functions
import subprocess # To start a shell command
import time  # Module for handling time-related operations

# Handy little enumeration to make code more readable

class Band:
    Delta = 0
    Theta = 1
    Alpha = 2
    Beta = 3

""" EXPERIMENTAL PARAMETERS """
# Modify these to change aspects of the signal processing

# Length of the EEG data buffer (in seconds)
# This buffer will hold last n seconds of data and be used for calculations
BUFFER_LENGTH = 5

# Length of the epochs used to compute the FFT (in seconds)
EPOCH_LENGTH = 1

# Amount of overlap between two consecutive epochs (in seconds)
OVERLAP_LENGTH = 0.8

# Amount to 'shift' the start of each next consecutive epoch
SHIFT_LENGTH = EPOCH_LENGTH - OVERLAP_LENGTH

# Index of the channel(s) (electrodes) to be used
# 0 = left ear, 1 = left forehead, 2 = right forehead, 3 = right ear
INDEX_CHANNEL = [2]

SILENCE = 10 # in seconds. Change this line of code to automatically update with LLM output

ALPHA_RELAXATION_THRESHOLD = 0.5
BETA_CONCENTRATION_THRESHOLD = 0.4
THETA_RELAXATION_THRESHOLD = 0.5

# Initialize metrics storage to calculate the mean over the SILENCE period
alpha_metrics = []
theta_metrics = []
beta_metrics = []

# Time tracking
start_time = time.time()
current_time = start_time

if __name__ == "__main__":

    subprocess.Popen(["muselsl", "stream"], shell=False) # Start the stream in a subprocess

    """ 1. CONNECT TO EEG STREAM """

    # Wait for the EEG stream to be found
    while True:
        print('Looking for an EEG stream...')
        streams = resolve_byprop('type', 'EEG', timeout=2)
        if len(streams) > 0:
            print('EEG stream found!')
            break
        else:
            print('No EEG stream found. Retrying in 2 seconds...')
            time.sleep(2)  # Wait for 2 seconds before retrying

    # Set active EEG stream to inlet and apply time correction
    print("Start acquiring data")
    inlet = StreamInlet(streams[0], max_chunklen=12)
    eeg_time_correction = inlet.time_correction()

    # Get the stream info and description
    info = inlet.info()
    description = info.desc()

    # Get the sampling frequency
    # This is an important value that represents how many EEG data points are
    # collected in a second. This influences our frequency band calculation.
    # for the Muse 2016, this should always be 256
    fs = int(info.nominal_srate())

    """ 2. INITIALIZE BUFFERS """

    # Initialize raw EEG data buffer
    eeg_buffer = np.zeros((int(fs * BUFFER_LENGTH), 1))
    filter_state = None  # for use with the notch filter

    # Compute the number of epochs in "buffer_length"
    n_win_test = int(np.floor((BUFFER_LENGTH - EPOCH_LENGTH) /
                              SHIFT_LENGTH + 1))

    # Initialize the band power buffer (for plotting)
    # bands will be ordered: [delta, theta, alpha, beta]
    band_buffer = np.zeros((n_win_test, 4))

    """ 3. GET DATA """

    # The try/except structure allows to quit the while loop by aborting the
    # script with <Ctrl-C>
    print('Press Ctrl-C in the console to break the while loop.')

    try:
        # The following loop acquires data, computes band powers, 
        #and calculates neurofeedback metrics based on those band powers
        while True:

            """ 3.1 ACQUIRE DATA """
            # Obtain EEG data from the LSL stream
            eeg_data, timestamp = inlet.pull_chunk(
                timeout=1, max_samples=int(SHIFT_LENGTH * fs))

            # Only keep the channel we're interested in
            ch_data = np.array(eeg_data)[:, INDEX_CHANNEL]

            # Update EEG buffer with the new data
            eeg_buffer, filter_state = utils.update_buffer(
                eeg_buffer, ch_data, notch=True,
                filter_state=filter_state)

            """ 3.2 COMPUTE BAND POWERS """
            # Get newest samples from the buffer
            data_epoch = utils.get_last_data(eeg_buffer,
                                             EPOCH_LENGTH * fs)

            # Compute band powers
            band_powers = utils.compute_band_powers(data_epoch, fs)
            band_buffer, _ = utils.update_buffer(band_buffer, band_powers)

            # Compute the average band powers for all epochs in buffer
            # This helps to smooth out noise
            smooth_band_powers = np.mean(band_buffer, axis=0)

            print('Delta: ', band_powers[0, Band.Delta], ' Theta: ', band_powers[0, Band.Theta],
                  ' Alpha: ', band_powers[0, Band.Alpha], ' Beta: ', band_powers[0, Band.Beta])


            """ 3.3 COMPUTE NEUROFEEDBACK METRICS """
            # These metrics could also be used to drive brain-computer interfaces

            # Alpha/Delta Protocol:
            # Simple redout of alpha power, divided by delta waves in order to rule out noise
            alpha_metric = smooth_band_powers[Band.Alpha] / \
                smooth_band_powers[Band.Delta]
            print('Alpha Relaxation: ', alpha_metric)

            # Beta Protocol:
            # Beta waves have been used as a measure of mental activity and concentration
            # This beta over theta ratio is commonly used as neurofeedback for ADHD
            beta_metric = smooth_band_powers[Band.Beta] / \
                 smooth_band_powers[Band.Theta]
            print('Beta Concentration: ', beta_metric)

            # Alpha/Theta Protocol:
            # This is another popular neurofeedback metric for stress reduction
            # Higher theta over alpha is supposedly associated with reduced anxiety
            theta_metric = smooth_band_powers[Band.Theta] / \
                 smooth_band_powers[Band.Alpha]
            print('Theta Relaxation: ', theta_metric)

            """ 3.4 RETURN MENTAL STATE"""
            # Given a SILENCE parameter fed from the LLM, calculates the predominant 
            # mental state during this period and returns 3 mental states: relaxed, focused, none

            # Store metrics
            alpha_metrics.append(alpha_metric)
            theta_metrics.append(theta_metric)
            beta_metrics.append(beta_metric)

            # Update current time
            current_time = time.time()

            # Check if SILENCE period has elapsed
            if current_time - start_time >= SILENCE:
                # Compute mean of the metrics over the SILENCE period
                mean_alpha = np.mean(alpha_metrics)
                mean_theta = np.mean(theta_metrics)
                mean_beta = np.mean(beta_metrics)

                # Determine mental state
                if (mean_alpha + mean_theta) > (ALPHA_RELAXATION_THRESHOLD + THETA_RELAXATION_THRESHOLD):
                    mental_state = "Relaxed"
                elif mean_beta > BETA_CONCENTRATION_THRESHOLD:
                    mental_state = "Focused"
                else:
                    mental_state = "None"

                # Reset metrics storage and start time for the next period
                alpha_metrics = []
                theta_metrics = []
                beta_metrics = []
                start_time = time.time()

                print(f"Mental State: {mental_state}")

    except KeyboardInterrupt:
        print('Closing!')

Looking for an EEG stream...
EEG stream found!
Start acquiring data
Searching for Muses, this may take up to 10 seconds...
Press Ctrl-C in the console to break the while loop.
Delta:  0.516262497941596  Theta:  -0.13212317043902633  Alpha:  -0.46664600185241956  Beta:  -0.61461348714604
Alpha Relaxation:  -0.9038928911416118
Beta Concentration:  4.651822122522247
Theta Relaxation:  0.2831336171627831
Mental State: Relaxed
Delta:  0.7562742243209183  Theta:  0.29874354398159  Alpha:  -0.097880981498053  Beta:  -0.21878851918052175
Alpha Relaxation:  -0.44362333398659676
Beta Concentration:  -5.001801332018181
Theta Relaxation:  -0.29515041522669166
Mental State: Relaxed
Delta:  0.782035995704329  Theta:  0.32754685874517836  Alpha:  -0.06293224062770915  Beta:  -0.09053364037868718
Alpha Relaxation:  -0.30539645469404486
Beta Concentration:  -1.8696821366076795
Theta Relaxation:  -0.7875686792117751
Mental State: Relaxed
Delta:  0.527474060038088  Theta:  0.041690684535622885  Alpha:  -

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

Disconnected.


2024-03-26 11:24:44.283 (2549.299s) [R_Muse          ]      data_receiver.cpp:344    ERR| Stream transmission broke off (Input stream error.); re-connecting...
